In [23]:
import requests
import json
import datetime
import pandas as pd

# ---------------------- КОНСТАНТЫ ----------------------
TOKEN = "c611c2bab48335e36a4b59be460c57d2BF8416B73C4A65F2B8A88A5848E97CD4471F14C6"
BASE_URL = "https://hst-api.wialon.host/wialon/ajax.html"

# ---------------------- ФУНКЦИИ ----------------------
def login(token):
    """
    Авторизация по токену. Возвращает SID (session ID).
    """
    resp = requests.get(
        BASE_URL,
        params={
            "svc": "token/login",
            "params": json.dumps({"token": token})
        }
    )
    data = resp.json()
    if "eid" not in data:
        raise RuntimeError(f"Ошибка при логине: {data}")
    return data["eid"]


def get_items(sid, item_type, flags):
    """
    Получает список объектов (юнитов или ресурсов).
    """
    resp = requests.get(
        BASE_URL,
        params={
            "svc": "core/search_items",
            "params": json.dumps({
                "spec": {
                    "itemsType": item_type,
                    "propName": "sys_name",
                    "propValueMask": "*",
                    "sortType": "sys_name"
                },
                "force": 1,
                "flags": flags,
                "from": 0,
                "to": 0
            }),
            "sid": sid
        }
    )
    data = resp.json()
    return data.get("items", [])


def get_raw_messages(sid, unit_id, day_from_ts, day_to_ts):
    """
    Запрашивает все сообщения (messages) для данного unit_id в заданном интервале.
    Возвращает список «сырых» JSON-объектов message.
    """
    resp = requests.get(
        BASE_URL,
        params={
            "svc": "messages/load_interval",
            "params": json.dumps({
                "itemId": unit_id,
                "timeFrom": day_from_ts,
                "timeTo": day_to_ts,
                "flags": 0x1,
                "flagsMask": 0,
                "loadCount": 0xFFFFFFFF
            }),
            "sid": sid
        }
    )
    data = resp.json()
    return data.get("messages", [])


# ---------------------- MAIN ----------------------
if __name__ == "__main__":
    # 1) Авторизуемся и получаем SID
    SID = login(TOKEN)

    # 2) Получаем список юнитов
    units = get_items(SID, "avl_unit", flags=1)
    if not units:
        print("❌ Юниты не найдены или нет прав доступа.")
        exit(1)

    # Выберем один unit_id для примера (например, четвёртый в списке)
    unit_id = units[3]["id"]
    unit_name = units[3]["nm"]

    # 3) Определяем timestamps для «сегодня» (00:00 и 23:59:59 UTC)
    today = datetime.date.today()
    day_from = datetime.datetime.combine(today, datetime.time.min)
    day_to   = datetime.datetime.combine(today, datetime.time.max)
    day_from_ts = int(day_from.timestamp())
    day_to_ts   = int(day_to.timestamp())

    # 4) Запрашиваем «сырые» сообщения для выбранного юнита
    raw_messages = get_raw_messages(SID, unit_id, day_from_ts, day_to_ts)
    if not raw_messages:
        print("ℹ️ Сообщений за этот день не найдено.")
        exit(0)

    # 5) Берём каждое 10-е сообщение (начиная от 10-го: индексы 9, 19, 29, ...)
    step_messages = raw_messages[9::10]

    # 6) Извлекаем поля (t, pos.y, pos.x, pos.s) для выбранных сообщений
    records = []
    for msg in step_messages:
        pos = msg.get("pos") or {}   # если pos==None → заменим на пустой словарь
        speed = pos.get("s", None)   # может быть None, если ключа «s» нет
        lat   = pos.get("y", None)
        lon   = pos.get("x", None)

        t_unix = msg.get("t", None)
        if isinstance(t_unix, int):
            t_dt = datetime.datetime.utcfromtimestamp(t_unix).strftime("%Y-%m-%d %H:%M:%S")
        else:
            t_dt = None

        records.append({
            "t (UTC)": t_dt,
            "lat": lat,
            "lon": lon,
            "speed (pos['s'])": speed
        })

    df = pd.DataFrame(records)

    # 7) Выводим таблицу с каждым 10-м сообщением
    print(f"\nСообщения юнита «{unit_name}» (id={unit_id}) – каждое 10-е за сегодня:\n")
    if df.empty:
        print("ℹ️ Нет сообщений на позициях 10, 20, 30 и т.д.")
    else:
        print(df.to_string(index=False))



Сообщения юнита «352 Балгожа» (id=27910469) – каждое 10-е за сегодня:

            t (UTC)       lat       lon  speed (pos['s'])
2025-06-03 19:09:38 53.234944 63.599540               0.0
2025-06-03 19:17:38 53.234944 63.599540               0.0
2025-06-03 19:25:38 53.234944 63.599540               0.0
2025-06-03 19:33:38 53.234944 63.599540               0.0
2025-06-03 19:41:38 53.234944 63.599540               0.0
2025-06-03 19:49:44       NaN       NaN               NaN
2025-06-03 19:59:38 53.234944 63.599540               0.0
2025-06-03 20:07:38 53.234944 63.599540               0.0
2025-06-03 20:15:38 53.234944 63.599540               0.0
2025-06-03 20:23:38 53.234944 63.599540               0.0
2025-06-03 20:31:38 53.234944 63.599540               0.0
2025-06-03 20:39:44       NaN       NaN               NaN
2025-06-03 20:49:38 53.234944 63.599540               0.0
2025-06-03 20:57:38 53.234944 63.599540               0.0
2025-06-03 21:05:38 53.234944 63.599540               0.0
